### Search Engine with Tools and Agents 

In [5]:
## Arxiv research
## Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [6]:
## Using inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [7]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arx = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arx.name

'arxiv'

In [ ]:
tools = [wiki,arx]

In [18]:
## Custom tools[Rag tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq

In [12]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model = "gemma:2b"))
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000249BA6156F0>, search_kwargs={})

In [15]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,"Langsmith search", "Search any information about Langsmith")
retriever_tool.name

'Langsmith search'

In [16]:
tools = [wiki, arx, retriever_tool]

In [17]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Langchain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Langsmith search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000249A02E04C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000249BA6156F0>, search_kwargs={}),

In [19]:
## Run all this tools with Agents and LLM models

## Tools, LLM --> AgentExecutor
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key = groq_api_key, model="llama3-8b-8192")

In [20]:
## Prompt Template

from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [21]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [22]:
## Agents

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [23]:
## AGent executor 
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [24]:
agent_executor.invoke({
    "input":"Tell me about Langsmith"
})



> Entering new AgentExecutor chain...

Invoking: `Langsmith search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Get started by adding tracing to your application.
Create dashboards to view key metrics like RPS, error rates and costs.

Evals​
The quality and development speed of AI applications depends on high-quality evaluation datasets and metrics to test and optimize your applications on. The LangSmith SDK and UI make building and running high-quality evaluations easy.

Get started by creating your first evaluation.
Quickly assess the performance of your application using our off-the-shelf evaluators as a starting point.
Analyze results of evaluations in the LangSmith UI and compare results over time.
Easily collect human feedback on your data to improve your application.

Prompt Engineering​
While traditional software applications are built by writing code, AI applications involve writing prompts to instruct the LLM on what to do. LangSmith provide

{'input': 'Tell me about Langsmith',
 'output': "It looks like the tool call yielded some text about Langsmith. Based on the text, Langsmith is a platform for building production-grade LLM (Large Language Model) applications. It provides features for observability, evaluation, and prompt engineering to help developers ship their applications quickly and with confidence.\n\nSome key features of Langsmith include:\n\n* Observability: Langsmith allows developers to closely monitor and evaluate their application, with features like tracing, metrics, dashboards, and alerts.\n* Evaluation: Langsmith enables developers to evaluate their application over production traffic, scoring its performance and collecting human feedback on the data.\n* Prompt Engineering: Langsmith provides tools for iterating on prompts, with automatic version control and collaboration features.\n\nIt seems like Langsmith is a powerful platform for building and deploying AI applications, and it's great that you were ab

In [25]:
agent_executor.invoke({
    "input":"WHat is machine learning"
})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witBased on the output from the tool call, I'll provide a concise summary:

Machine learning is a subfield of artificial intelligence that involves developing and studying algorithms that can learn from data and make predictions or decisions without being explicitly programmed. These algorithms can analyze data, identify patterns, and make informed decisions, enabling applications in areas such as image and speech recognition, natural language processing, and recommender systems.

Would you like to know more about a specific aspect of machine learning, or is there another topic you'd like to explore?

> Finished chain.


{'input': 'WHat is machine learning',
 'output': "Based on the output from the tool call, I'll provide a concise summary:\n\nMachine learning is a subfield of artificial intelligence that involves developing and studying algorithms that can learn from data and make predictions or decisions without being explicitly programmed. These algorithms can analyze data, identify patterns, and make informed decisions, enabling applications in areas such as image and speech recognition, natural language processing, and recommender systems.\n\nWould you like to know more about a specific aspect of machine learning, or is there another topic you'd like to explore?"}

In [27]:
agent_executor.invoke({
    "input":"WHat is the paper 1706.03762 all about "
})  



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cBased on the output from the "arxiv" tool, it appears that the paper "1706.03762" is a research paper titled "Attention Is All You Need" by Ashish Vaswani et al. The paper was published on August 2, 2023, and it seems to be discussing sequence transduction models. 

Here's the text-based response:

The paper "Attention Is All You Need" by Ashish Vaswani et al. was published on August 2, 2023. The paper discusses sequence transduction models.

> Finished chain.


{'input': 'WHat is the paper 1706.03762 all about ',
 'output': 'Based on the output from the "arxiv" tool, it appears that the paper "1706.03762" is a research paper titled "Attention Is All You Need" by Ashish Vaswani et al. The paper was published on August 2, 2023, and it seems to be discussing sequence transduction models. \n\nHere\'s the text-based response:\n\nThe paper "Attention Is All You Need" by Ashish Vaswani et al. was published on August 2, 2023. The paper discusses sequence transduction models.'}